In [10]:
#read in file
df = spark.read.csv("C:/Users/290002494/Documents/Python Scripts/Data Science/Pokemon.csv", header=True, inferSchema=True)

In [11]:
df.groupBy("Stage").count().show()

+-----+-----+
|Stage|count|
+-----+-----+
|    1|   79|
|    3|   16|
|    2|   56|
+-----+-----+



In [12]:
def large_data_plot(data_frame, column_sample_by = "", num_points = 100, sample_ratio = [], cols = []):
    
    #if column_sample_by is blank then just sample based on num_points
    
    #sample_ratio should add up to 1
    #num_points is the total number of points you want in the final dataset
    #column_sample_by is the column you want to stratify
    #cols is to select only necessary columns
    
    #Method is robust to theoretically work with any number of categories binary to many
    
    #import necessary modules
    import numpy as np
    import itertools
    from scipy.optimize import linprog


    if column_sample_by != "":

        unique_value_length = len(sample_ratio)

        #get our totals of each by variable occurence
        pd_frame = data_frame.select(column_sample_by).groupBy(column_sample_by).count().orderBy(column_sample_by).select(column_sample_by,"count").toPandas()

        #get list of unique values and their total count
        sampling_variable = list(pd_frame[column_sample_by])
        sampling_totals = list(pd_frame["count"])

        #get all possible constraint combinations
        columns = []
        for i in range(unique_value_length):
            if i < 2:
                pass
            else:
                columns.append(list(itertools.combinations(range(unique_value_length),i)))

        columns = [part for item in columns for part in item]
        #if binary make the list empty
        if unique_value_length < 3:
            columns = []

        ###set up format for simplex method
        #3 parts to our matrix

        #main constraint => + 1
        #relationship inequalities => n(n-1)/2. number of rows for relationship constraints
        #combinations => len(columns)

        #add all 3 to get the number of rows for our matrix
        #length of each array needs to add unique_value_length because each inequality has a slack variable

        sum_formula = ((unique_value_length)*(unique_value_length -1))/2
        num_arrays_needed = int(sum_formula + len(columns) + 1)
        length_of_each_array = unique_value_length + num_arrays_needed

        #initialize our linear program
        program_array = np.zeros((num_arrays_needed, length_of_each_array))


        #minimization function
        program_array[0,range(unique_value_length)] = -1

        c = program_array[0, :].copy()


        #reset our first row
        program_array[0,range(unique_value_length)] = 1   
        program_array[0, unique_value_length] = 1 #the slack variable in the first/main inequality



        #make the constraints

        for i in range(1,num_arrays_needed): #skip the first row as it was a special inequality that we already made
            #add slack terms
            program_array[i,i+unique_value_length] = 1

            if i in list(range(1,len(columns) + 1)):
                program_array[i,columns[i-1]] = 1
                #need to make relationship inequalities for the ones with 2 value column lengths
                if len(columns[i-1]) == 2:
                    program_array[i+len(columns),columns[i-1]] = 1

        if unique_value_length < 3:
            program_array[1,0:2] = 1


        #make b matrix
        b = np.zeros(num_arrays_needed)
        b[0] = num_points

        for i in range(len(b)):
            if i not in list(range(0,len(columns)+1)):
                b[i] = 0
            elif i == 0:
                pass
            else:
                value_holder = sum([sample_ratio[i] for i in columns[i-1]])
                b[i] = num_points*value_holder

        #make the final few rows in the matrix A...
        for i in range(len(b)):
            if b[i] == 0:#i not in list(range(0,len(columns)+1)):
                index_value_last = np.nonzero(program_array[i,:unique_value_length])[0][-1]
                index_value_first = np.nonzero(program_array[i,:unique_value_length])[0][0]
                #add the constraint values correctly
                program_array[i, index_value_last] = \
                -(sample_ratio[index_value_first])/(sample_ratio[index_value_last])

        #print(c)
        #print(np.round(program_array,2))
        print(b)


        #set up and run simplex method
        bound_list = []

        #set up bounds for slack variables of the inequalities
        for i in range(length_of_each_array):

            if i < unique_value_length:

                #can't go over the maximum number of points in that category that exist
                bound_list.append((1,min(num_points*sample_ratio[i], sampling_totals[i])))

            elif (i >= unique_value_length) & (i < 1 + unique_value_length + len(columns)):
                bound_list.append((0,10000))
            else:
                bound_list.append((0,1))
        #make a tuple 
        tuple_bounds = tuple(bound_list)

        #solve our equations
        res = linprog(c, A_eq = program_array, b_eq = b, 
                      bounds = tuple_bounds,
                 method = "simplex")
        print(res.x[:unique_value_length])

        #return our values
        numbers_per_category =  np.round(res.x[:unique_value_length],0)
        percentages = np.round(numbers_per_category/sampling_totals,2)

        #now retrieve the dataset
        fractions = {}
        for i in range(unique_value_length):
            fractions[sampling_variable[i]] = percentages[i]


        sampled_df = data_frame.sampleBy(column_sample_by, fractions = fractions)
    
    else:
        percentages = np.round(num_points/data_frame.count(),2)
        sampled_df = data_frame.sample(False, percentages)
        print(percentages)
        print(sampled_df.count())
        
    #select necessary columns
    if cols == []:
        pass
    else:
        #fix columns with spaces or periods in their names
        new_cols = []
        for i in sampled_df.columns:
            sampled_df = sampled_df.withColumnRenamed(i, i.replace(" ", "_").replace(".", ""))
        col_list = cols
        #fix in case variables have a space in their name only for the variables required
        col_list = [i.replace(" ", "_").replace(".", "") for i in col_list]
        sampled_df.select([col for col in sampled_df.columns if col in col_list])
    
    return sampled_df.toPandas()

In [13]:
large_data_plot(data_frame = df, num_points = 100, column_sample_by = "Stage", sample_ratio = [.3,.4,.2], cols = []).to_csv("test.csv")

[100.  70.  50.  60.   0.   0.   0.]
[24. 32. 16.]


In [ ]:
! bokeh serve --show Simplex_Dashboard.ipynb --port 5004
#runs dashboard